# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using PseudoPotentialData

pseudopotentials = PseudoFamily("dojo.nc.sr.pbesol.v0_4_1.standard.upf")
model = model_DFT(bulk(:Si); functionals=PBEsol(), pseudopotentials)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397881619231                   -0.90    5.2   27.6ms
  2   -8.400248865337       -2.63       -1.74    1.0   21.5ms
  3   -8.400406829544       -3.80       -2.96    1.5   22.7ms
  4   -8.400427831736       -4.68       -2.97    3.2   27.3ms
  5   -8.400427922022       -7.04       -3.02    1.0   45.2ms
  6   -8.400428150008       -6.64       -4.61    1.0   19.5ms
  7   -8.400428155806       -8.24       -4.44    2.2   23.4ms
  8   -8.400428156249       -9.35       -5.16    1.0   20.0ms
  9   -8.400428156275      -10.58       -6.47    1.2   20.9ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -8.397804305552                   -0.90           5.5    488ms
  2   -8.400384184276       -2.59       -1.79   0.80    2.0    527ms
  3   -8.400422694129       -4.41       -3.01   0.80    1.0    180ms
  4   -8.400428106287       -5.27       -3.42   0.80    2.2   20.3ms
  5   -8.400428152860       -7.33       -4.64   0.80    1.2   16.8ms
  6   -8.400428156243       -8.47       -5.65   0.80    2.8   66.8ms
  7   -8.400428156276      -10.48       -5.85   0.80    2.2   20.8ms
  8   -8.400428156277      -12.40       -6.93   0.80    1.0   16.5ms


## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   +0.680406353674                   -1.08    2.85s
  2   -1.846083895866        0.40       -0.65    156ms
  3   -3.524403721224        0.22       -0.41   40.0ms
  4   -4.315698887437       -0.10       -0.49   40.1ms
  5   -5.952933983739        0.21       -0.51   40.2ms
  6   -6.849331628373       -0.05       -0.72   68.0ms
  7   -7.798709647502       -0.02       -0.88   40.3ms
  8   -7.881662560143       -1.08       -1.48   29.3ms
  9   -8.146333635650       -0.58       -1.69   29.2ms
 10   -8.179099215309       -1.48       -2.01   34.3ms
 11   -8.319926433093       -0.85       -1.82   29.5ms
 12   -8.359517497653       -1.40       -2.29   29.3ms
 13   -8.383124793950       -1.63       -2.43   29.2ms
 14   -8.389905762043       -2.17       -2.56   33.3ms
 15   -8.396345183739       -2.19       -2.89   30.7ms
 16   -8.397549306578       -2.92       -2.85   29.0ms
 17   -8.39

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397819614074                   -0.90    5.2   27.1ms


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -8.400427986180                   -1.79    25.6s
  2   -8.400428156277       -6.77       -4.03    2.18s
  3   -8.400428156277      -14.75       -7.83   94.2ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.1599198026355412e-7
|ρ_newton - ρ_scfv| = 4.3678437666398486e-8
|ρ_newton - ρ_dm|   = 9.62393857404865e-7
